## Typing in Agent Building

### Will's Take on Typing

* Hints not mandatory in Python, but strongly recommended
* "Any" as an escape hatch if needed, but "Union" preferable
* Ensures reliable composability
* Enable Pylance (Pyright) in IDE
* Linting for code quality (Pylance, Ruff)
* Great for LLM-assisted development, many IDEs will show linter errors to LLMs, catches many bugs early

In [6]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [1]:
def cumulative_reward(daily_returns):

    acc = 1
    totals = []
    for ret in daily_returns:
        acc *= ret
        totals.append(acc)
    
    return totals

rets = [1.1, 1.5, 2]
cumulative_reward(rets)

[1.1, 1.6500000000000001, 3.3000000000000003]

In [7]:
from openai import OpenAI

prompt = """
Value yesterday: 500

Value today: 760

Return the growth multiplier (number only).
"""

oai = OpenAI()

response = oai.chat.completions.create(
    model = 'gpt-4.1-nano', 
    messages=[{"role": "user", "content": prompt}],
)

mult = response.choices[0].message.content

print(f"Growth Multiplier: {mult}")

Growth Multiplier: 1.52


In [8]:
rets = [1, 3, 4]
rets.append(mult) #type: ignore
cumulative_reward(rets)

[1, 3, 12, '1.521.521.521.521.521.521.521.521.521.521.521.52']

In [9]:
def cumulative_returns(daily_returns: list[float]) -> list[float]:
    """
    Return the running product of daily returns.
    e.g. [1.1, 1.03, 1.04]  ->  [1.1, 1.133, 1.1783]
    """
    acc = 1
    totals = []
    for ret in daily_returns:
        acc *= ret
        totals.append(acc)
    return totals

rets = [1.1, 1.03, 1.04]
cumulative_returns(rets)

[1.1, 1.1330000000000002, 1.1783200000000003]

In [ ]:
rets.append(mult) #type: ignore
cumulative_returns(rets)

TypeError: can't multiply sequence by non-int of type 'float'

## Pytest

In [20]:
import pytest


def test_cumulative_returns_basic():
    """Test basic cumulative returns calculation"""
    result = cumulative_returns([1.1, 1.03, 1.04])
    expected = [1.1, 1.133, 1.17832]
    for actual, expected_val in zip(result, expected):
        assert actual == pytest.approx(expected_val, rel=1e-10)

def test_cumulative_returns_single():
    """Test with single return value"""
    assert cumulative_returns([1.5]) == [1.5]

def test_cumulative_returns_empty():
    """Test with empty list"""
    assert cumulative_returns([]) == []

def test_cumulative_returns_identity():
    """Test with identity returns (1.0)"""
    assert cumulative_returns([1.0, 1.0, 1.0]) == [1.0, 1.0, 1.0]


def test_cumulative_returns_negative():
    """Test with negative returns"""
    result = cumulative_returns([0.9, 0.8])
    expected = [0.9, 0.72]
    for actual, expected_val in zip(result, expected):
        assert actual == pytest.approx(expected_val, rel=1e-10)


def test_cumulative_returns_mixed():
    """Test with mixed positive and negative returns"""
    result = cumulative_returns([1.1, 0.9, 1.2])
    expected = [1.1, 0.99, 1.188]
    for actual, expected_val in zip(result, expected):
        assert actual == pytest.approx(expected_val, rel=1e-10)

def test_cumulative_returns_type_error():
    """ Test with string input """

    with pytest.raises((TypeError)):
        cumulative_returns(["1.1", "1.03", "1.04"]) 

def test_cumulative_returns_mixed_error():
    """ Test with type error"""

    with pytest.raises(TypeError):
        cumulative_returns([1.1, "1.03", 1.04])

def run_tests():
    test_functions = [
        test_cumulative_returns_basic,
        test_cumulative_returns_single, 
        test_cumulative_returns_empty,
        test_cumulative_returns_identity,
        test_cumulative_returns_negative,
        test_cumulative_returns_mixed,
        test_cumulative_returns_type_error, 
        test_cumulative_returns_mixed_error
    ]

    passed = 0
    failed = 0


    for test_func in test_functions:
        try:
            test_func()
            print(f"✅ {test_func.__name__}: PASSED")
            passed += 1
        except Exception as e:
            print(f"❌ {test_func.__name__}: FAILED - {str(e)}")
            failed += 1

    print(f"\n{'='*50}")
    print(f"Results: {passed} passed, {failed} failed")

    if failed == 0:
        print("🎉 All tests passed!")
    
    return failed == 0

run_tests()



✅ test_cumulative_returns_basic: PASSED
✅ test_cumulative_returns_single: PASSED
✅ test_cumulative_returns_empty: PASSED
✅ test_cumulative_returns_identity: PASSED
✅ test_cumulative_returns_negative: PASSED
✅ test_cumulative_returns_mixed: PASSED
✅ test_cumulative_returns_type_error: PASSED
✅ test_cumulative_returns_mixed_error: PASSED

Results: 8 passed, 0 failed
🎉 All tests passed!


True

In [ ]:
from pydantic import BaseModel
import instructor

class GrowthMultiplier(BaseModel):
    growth_multiplier: float

prompt = """
Value yesterday: 500

Value today: 760

Return the growth multiplier (number only).
"""

oai = OpenAI()

instructor_oai = instructor.from_openai(oai)

response = instructor_oai.chat.completions.create(
    model = "gpt-4.1-nano", 
    messages = [{"role": "user",  "content": prompt}], 
    response_model= GrowthMultiplier,
)

print(f"Growth Multipler: {response.growth_multiplier}")


Growth Multipler: 1.52


Example of a bug that is hard to detect in the wild i guess? 

In [23]:
def calculator(expression: str) -> str:
    """Evaluates a single line of Python math expression. No imports or variables allowed.

    Args:
        expression (str): A mathematical expression using only numbers and basic operators (+,-,*,/,**,())

    Returns:
        The result of the calculation or an error message

    Examples:
        "2 + 2" -> "4"
        "3 * (17 + 4)" -> "63"
        "100 / 5" -> "20.0"
    """
    allowed = set("0123456789+-*/.() ")
    
    if not all(c in allowed for c in expression):
        return "Error: Invalid characters in expression"
    
    try:
        # eval is a dangerous function, use with caution
        result = eval(expression, {"__builtins__": {}}, {})
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}" 
    
print(calculator("2 + 2"))
print(calculator("3 x (17 + 4)"))
print(calculator("(62565374 + 265345356) / 425634563456"))


4
Error: Invalid characters in expression
0.0007704043753812719


In [26]:
# openai agents sdk
# uv add openai-agents

import asyncio
import nest_asyncio
nest_asyncio.apply() #

from agents import Agent, Runner, function_tool

def calculator(expression: str) -> str:
    """Evaluates a single line of Python math expression. No imports or variables allowed.

    Args:
        expression (str): A mathematical expression using only numbers and basic operators (+,-,*,/,**,())

    Returns:
        The result of the calculation or an error message

    Examples:
        "2 + 2" -> "4"
        "3 * (17 + 4)" -> "63"
        "100 / 5" -> "20.0"
    """
    allowed = set("0123456789+-*/.() ")
    if not all(c in allowed for c in expression):
        return "Error: Invalid characters in expression"
    
    try:
        # eval is a dangerous function, use with caution
        result = eval(expression, {"__builtins__": {}}, {})
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}" 
    
print(calculator("2 + 2"))
print(calculator("3 x (17 + 4)"))
print(calculator("(62565374 + 265345356) / 425634563456"))

prompt = """
What is (62565374 + 265345356) / 425634563456?
"""


@function_tool
async def calculator_tool(expression: str) -> str:
    """Evaluates a single line of Python math expression. No imports or variables allowed.

    Args:
        expression (str): A mathematical expression using only numbers and basic operators (+,-,*,/,**,())

    Returns:
        The result of the calculation or an error message

    Examples:
        "2 + 2" -> "4"
        "3 * (17 + 4)" -> "63"
        "100 / 5" -> "20.0"
    """
    return calculator(expression)

agent = Agent(
    model="gpt-4.1-nano",
    name="calculator_agent",
    tools=[calculator_tool],
)
print("---")

result = Runner.run_sync(agent, prompt)
print(result.final_output)

agent = Agent(
    model="gpt-4.1-nano",
    name="calculator_agent",
    tools=[],
)
print("---")
result = Runner.run_sync(agent, prompt)
print(result.final_output)

4
Error: Invalid characters in expression
0.0007704043753812719
---


/Users/vijayravichander/Code/rl-agents-intro-will/rl-agents-intro-will/.venv/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:963: RuntimeWarning: coroutine 'AgentRunner.run' was never awaited
  args = tuple(self._resolve_forward_ref(a) if isinstance(a, ForwardRef) else a for a in args)


The result of \(\frac{62565374 + 265345356}{425634563456}\) is approximately \(0.0007704\).
---
Let's calculate the value step by step:

1. Add 62,565,374 and 265,345,356:
\[
62,565,374 + 265,345,356 = 327,910,730
\]

2. Divide this sum by 425,634,563,456:
\[
\frac{327,910,730}{425,634,563,456} \approx 0.0000007695
\]

**Final answer: approximately 0.0000007695**.


### !!! Async Processing

In [27]:
prompts = []

search_words = ["San Franscisco", "New York City", "Dallas", "Chicago", "San Jose", "New Jersey", "Atlanta", "Arizona"]

for search in search_words:
    prompts.append(f"Top 10 places to rent as new grad computer science student in {search}")

['Top 10 places to rent as new grad computer science student in San Franscisco',
 'Top 10 places to rent as new grad computer science student in New York City',
 'Top 10 places to rent as new grad computer science student in Dallas',
 'Top 10 places to rent as new grad computer science student in Chicago',
 'Top 10 places to rent as new grad computer science student in San Jose',
 'Top 10 places to rent as new grad computer science student in New Jersey',
 'Top 10 places to rent as new grad computer science student in Atlanta',
 'Top 10 places to rent as new grad computer science student in Arizona']

In [29]:
class Place(BaseModel):
    location: str

class Places(BaseModel):
    locations: list[Place]


for prompt in prompts:
    response = instructor_oai.chat.completions.create(
        model = "gpt-4.1-nano", 
        messages = [{"role": "user", "content": prompt}], 
        response_model = Places
    )

    print(response.locations)




[Place(location='San Francisco')]
[Place(location='New York City')]
[Place(location='Dallas')]
[Place(location='Chicago')]
[Place(location='San Jose')]
[Place(location='Jersey City, NJ'), Place(location='Newark, NJ'), Place(location='Hoboken, NJ'), Place(location='Paterson, NJ'), Place(location='Elizabeth, NJ'), Place(location='Bayonne, NJ'), Place(location='Clifton, NJ'), Place(location='Trenton, NJ'), Place(location='Montclair, NJ'), Place(location='Union City, NJ')]
[Place(location='Atlanta, Georgia')]
[Place(location='Phoenix, Arizona'), Place(location='Tucson, Arizona'), Place(location='Mesa, Arizona'), Place(location='Chandler, Arizona'), Place(location='Gilbert, Arizona'), Place(location='Glendale, Arizona'), Place(location='Scottsdale, Arizona'), Place(location='Tempe, Arizona'), Place(location='Peoria, Arizona'), Place(location='Yuma, Arizona')]


In [31]:
from openai import AsyncOpenAI
import asyncio
import nest_asyncio
nest_asyncio.apply()

oai = AsyncOpenAI()
instructor_oai = instructor.from_openai(oai)

async def get_locations(keyword: str) -> list[Place]:
    response = await instructor_oai.chat.completions.create(
        model = "gpt-4.1-nano", 
        messages = [{"role": "user", "content": keyword}], 
        response_model=Places
    )

    return response.locations


async def main():
    tasks = [get_locations(keyword) for keyword in search_words]
    results = await asyncio.gather(*tasks)
    return results

results = asyncio.run(main())
print(results)

[[Place(location='San Francisco')], [Place(location='New York City')], [Place(location='Dallas')], [Place(location='Chicago')], [Place(location='San Jose')], [Place(location='New Jersey')], [Place(location='Atlanta')], [Place(location='Arizona')]]


In [32]:
## Without Instrcutor

from openai import AsyncOpenAI
import asyncio
import nest_asyncio
from typing import Union
nest_asyncio.apply()

oai = AsyncOpenAI()

async def get_locations(keyword: str) -> Union[str, None]:
    response = await oai.chat.completions.create(
        model = "gpt-4.1-nano", 
        messages = [{"role": "user", "content": keyword}], 
    )

    return response.choices[0].message.content
    


async def main():
    tasks = [get_locations(keyword) for keyword in search_words]
    results = await asyncio.gather(*tasks)
    return results

results = asyncio.run(main())
print(results)



["San Francisco is a vibrant city located in Northern California, renowned for its diverse culture, iconic landmarks, and innovative spirit. It is famous for the Golden Gate Bridge, Alcatraz Island, and its historic cable cars. The city is a hub for technology, with nearby Silicon Valley housing many leading tech companies. San Francisco also offers a rich arts scene, picturesque neighborhoods like Fisherman's Wharf and Chinatown, and beautiful natural surroundings such as parks and the nearby Marin Headlands. Whether you're interested in history, technology, or scenic views, San Francisco has something to offer.", 'New York City (NYC) is the largest city in the United States, located in the state of New York. Known for its vibrant culture, diverse population, and iconic landmarks, NYC is often referred to as "The Big Apple." The city comprises five boroughs: Manhattan, Brooklyn, Queens, The Bronx, and Staten Island.\n\nMajor attractions include:\n- Times Square\n- Central Park\n- Stat

In [33]:
for result in results:
    print(result)

San Francisco is a vibrant city located in Northern California, renowned for its diverse culture, iconic landmarks, and innovative spirit. It is famous for the Golden Gate Bridge, Alcatraz Island, and its historic cable cars. The city is a hub for technology, with nearby Silicon Valley housing many leading tech companies. San Francisco also offers a rich arts scene, picturesque neighborhoods like Fisherman's Wharf and Chinatown, and beautiful natural surroundings such as parks and the nearby Marin Headlands. Whether you're interested in history, technology, or scenic views, San Francisco has something to offer.
New York City (NYC) is the largest city in the United States, located in the state of New York. Known for its vibrant culture, diverse population, and iconic landmarks, NYC is often referred to as "The Big Apple." The city comprises five boroughs: Manhattan, Brooklyn, Queens, The Bronx, and Staten Island.

Major attractions include:
- Times Square
- Central Park
- Statue of Libe

In [34]:
## Semaphore Version to Rate Limit
async def get_locations_with_sem(keyword: str, semaphore: asyncio.Semaphore) -> Union[str, None]:

    async with semaphore:
        response = await oai.chat.completions.create(
            model = "gpt-4.1-nano", 
            messages = [{"role": "user", "content": f"Top 10 places to rent as new grad computer science student in {keyword}. Just the 10 locations"}], 
        )

        return response.choices[0].message.content
    
async def main():
    semaphore = asyncio.Semaphore(5)
    tasks = [get_locations_with_sem(keyword, semaphore) for keyword in search_words]
    results = await asyncio.gather(*tasks)
    return results

results = asyncio.run(main())
print(results)



['1. Inner Sunset  \n2. Mission District  \n3. SoMa (South of Market)  \n4. Hayes Valley  \n5. North Beach  \n6. Japantown  \n7. Tenderloin  \n8. Haight-Aechton  \n9. Richmond District  \n10. South of Market (SoMa)', '1. Astoria, Queens  \n2. Long Island City, Queens  \n3. Williamsburg, Brooklyn  \n4. DUMBO, Brooklyn  \n5. Bushwick, Brooklyn  \n6. Harlem, Manhattan  \n7. Washington Heights, Manhattan  \n8. East Village, Manhattan  \n9. Sunnyside, Queens  \n10. Financial District, Manhattan', '1. Uptown Dallas\n2. Deep Ellum\n3. Downtown Dallas\n4. Oak Lawn\n5. Bishop Arts District\n6. Lower Greenville\n7. Medical District\n8. Turtle Creek\n9. Fair Park Area\n10. Victory Park', '1. Near North Side (River North)  \n2. Lincoln Park  \n3. Lakeview (including Boystown)  \n4. South Loop  \n5. Wicker Park/Bucktown  \n6. West Loop (Fulton Market)  \n7. Hyde Park  \n8. Near West Side (Ukrainian Village, Little Italy)  \n9. Logan Square  \n10. Rogers Park', '1. Downtown San Jose  \n2. North San 

In [50]:

# Toolkit


def calculator(expression: str) -> str:
    """Evaluates a single line of Python math expression. No imports or variables allowed.

    Args:
        expression (str): A mathematical expression using only numbers and basic operators (+,-,*,/,**,())

    Returns:
        The result of the calculation or an error message

    Examples:
        "2 + 2" -> "4"
        "3 * (17 + 4)" -> "63"
        "100 / 5" -> "20.0"
    """
    allowed = set("0123456789+-*/.() ")
    if not all(c in allowed for c in expression):
        return "Error: Invalid characters in expression"
    
    try:
        # eval is a dangerous function, use with caution
        result = eval(expression, {"__builtins__": {}}, {})
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}" 

def secret_digit(secret: str) -> str:
    return "The secret number is 7"

async def execute_tool(func, args) -> str:
    return func(**args)

In [52]:

# Parsers


import re
import json

def parse_thinking_from_response(response: str) -> str | None:
    """Parse a thinking from a response."""
    # re.DOTALL is used to all \n inside the think tags
    # ? matches lazily meaning we pick the content inside the first <think> </think>
    thinking = re.search(r'<think>(.*?)</think>', response, re.DOTALL)
    if thinking:
        return thinking.group(1)
    return None

def parse_tool_from_response(response: str) -> dict | None:
    """Parse a tool from a response."""
    tool_call = re.search(r'<tool>(.*?)</tool>', response, re.DOTALL)
    if tool_call:
        return json.loads(tool_call.group(1))
    return None


def parse_answer_from_response(response: str) -> str | None:
    """Parse an answer from a response."""
    answer = re.search(r'<answer>(.*?)</answer>', response, re.DOTALL)
    if answer:
        return answer.group(1)
    return None

In [49]:

# Async Version of Tool Call

async def call_tool(tool_call: dict) -> str:
    """Call a tool with the given tool call."""

    if tool_call['name'] == 'calculator':
        return calculator(tool_call['args']['calculator'])

    elif tool_call['name'] == 'secret_digit':
        return secret_digit(tool_call['args']['secret'])

    else:
        return f"Error: Tool {tool_call['name']} not found"


In [51]:
from openai import AsyncOpenAI

oai = AsyncOpenAI()

async def agent_loop(question: str) -> Union[str, None]:

    system_prompt = """ 
    You are assistant who needs to find the secret number 
    You have access to the following tools:
    - calculator(calculator: str) -> str: Calculates the expressions. 
    - secret_digit(secret: str) -> str: Helps you find the secret number.

    You may call one tool per turn, for up to 10 turns, before giving your final answer.

    In each turn, you should respond in the following format:

    <think>
    [your thoughts here]
    </think>
    <tool>
    JSON with the following fields:
    - name: The name of the tool to call
    - args: A dictionary of arguments to pass to the tool (must be valid JSON)
    </tool>

    When you are done, give your final answer in the following format:

    <answer>
    [your final answer here]
    </answer>
    """

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question},
    ]

    turns = 0
    
    while True:
        turns += 1

        try:
            response = await oai.chat.completions.create(
                model="gpt-4.1-nano",
                messages=messages, # type: ignore
            )

            response = response.choices[0].message.content # type: ignore

            # parse for thinking, tool call, and/or answer 
            maybe_thinking = parse_thinking_from_response(response) # type: ignore
            maybe_tool_call = parse_tool_from_response(response) # type: ignore
            maybe_answer = parse_answer_from_response(response) # type: ignore

        except Exception as e:
            print(f"Error: {e}")
            retries += 1
            
        print("=== Turn", turns, "===")

        if maybe_thinking: # type: ignore
            thinking = maybe_thinking.strip()
            print(f"Thinking: {thinking}")

        if maybe_tool_call: # type: ignore
            tool_call = maybe_tool_call
            tool_result = await call_tool(tool_call)
            print(f"Tool call: {tool_call}")
            print(f"Tool result: {tool_result[:100]}")
            messages.append({"role": "user", "content": tool_result})

        elif maybe_answer: # type: ignore
            final_answer = maybe_answer
            return final_answer

        else:
            raise ValueError

async def main():
    prompts = ["Could you tell me what the secret number is?", "What’s the hidden number, please?", "What’s the hidden number, please?", "Reveal the secret number, would you?",
               "Which number are you keeping under wraps?", "I’m curious—what’s the concealed number?"]

    agent_calls = [agent_loop(prompt) for prompt in prompts]
    results = await asyncio.gather(*agent_calls)
    return results


results = asyncio.run(main());
results


=== Turn 1 ===
Thinking: I need more information or a hint to determine the secret number. Since I don't have any clues, I will try asking the secret_digit tool to directly find out the secret number.
Tool call: {'name': 'secret_digit', 'args': {'secret': ''}}
Tool result: The secret number is 7
=== Turn 1 ===
Thinking: I need more information or questions to determine the secret number. Since I can only use the tools provided, I should probably start by asking the tool that helps me identify the secret digit directly.
Tool call: {'name': 'secret_digit', 'args': {'secret': ''}}
Tool result: The secret number is 7
=== Turn 1 ===
Thinking: I need to gather some information or perform calculations to identify the secret number. Since no specific hints or equations are provided, I should perhaps attempt some initial guesswork or inquire about the secret number directly.
Tool call: {'name': 'secret_digit', 'args': {'secret': ''}}
Tool result: The secret number is 7
=== Turn 1 ===
Thinking: 

['\nThe secret number is 7.\n',
 '\nThe secret number is 7.\n',
 '\nThe secret number is 7.\n',
 '\nThe secret number is 7.\n',
 '\nThe secret number is 7.\n',
 '\nThe secret number is 7.\n']

In [39]:
# sync version of tool call 
def call_tool(tool_call: dict) -> str:
    """Call a tool with the given tool call."""

    if tool_call['name'] == 'calculator':
        return calculator(tool_call['args']['calculator'])

    elif tool_call['name'] == 'secret_digit':
        return secret_digit(tool_call['args']['secret'])

    else:
        return f"Error: Tool {tool_call['name']} not found"

In [48]:
from openai import OpenAI

oai = OpenAI()

def agent_loop(question: str) -> Union[str, None]:

    system_prompt = """ 
    You are assistant who needs to find the secret number 
    You have access to the following tools:
    - calculator(calculator: str) -> str: Calculates the expressions. 
    - secret_digit(secret: str) -> str: Helps you find the secret number.

    You may call one tool per turn, for up to 10 turns, before giving your final answer.

    In each turn, you should respond in the following format:

    <think>
    [your thoughts here]
    </think>
    <tool>
    JSON with the following fields:
    - name: The name of the tool to call
    - args: A dictionary of arguments to pass to the tool (must be valid JSON)
    </tool>

    When you are done, give your final answer in the following format:

    <answer>
    [your final answer here]
    </answer>
    """

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question},
    ]

    turns = 0
    
    while True:
        turns += 1

        try:
            response = oai.chat.completions.create(
                model="gpt-4.1-nano",
                messages=messages, # type: ignore
            )

            response = response.choices[0].message.content # type: ignore

            # parse for thinking, tool call, and/or answer 
            maybe_thinking = parse_thinking_from_response(response) # type: ignore
            maybe_tool_call = parse_tool_from_response(response) # type: ignore
            maybe_answer = parse_answer_from_response(response) # type: ignore

        except Exception as e:
            print(f"Error: {e}")
            retries += 1
            
        print("=== Turn", turns, "===")

        if maybe_thinking: # type: ignore
            thinking = maybe_thinking.strip()
            print(f"Thinking: {thinking}")

        if maybe_tool_call: # type: ignore
            tool_call = maybe_tool_call
            tool_result = call_tool(tool_call)
            print(f"Tool call: {tool_call}")
            print(f"Tool result: {tool_result[:100]}")
            messages.append({"role": "user", "content": tool_result})

        elif maybe_answer: # type: ignore
            print(f"Final Answer: {maybe_answer}")
            final_answer = maybe_answer
            return final_answer

        else:
            raise ValueError


prompts = ["Could you tell me what the secret number is?", "What’s the hidden number, please?", "What’s the hidden number, please?", "Reveal the secret number, would you?",
            "Which number are you keeping under wraps?", "I’m curious—what’s the concealed number?"]

agent_calls = [agent_loop(prompt) for prompt in prompts]

agent_calls

=== Turn 1 ===
Thinking: I need to gather some information or perform calculations to determine the secret number. Since I don't have any initial clues, I will start by requesting the secret digit directly to see if that gives me any insight.
Tool call: {'name': 'secret_digit', 'args': {'secret': ''}}
Tool result: The secret number is 7
=== Turn 2 ===
Thinking: Since you stated that the secret number is 7, I can verify this by using the secret_digit tool.
I will now call the secret_digit tool to confirm.
Tool call: {'name': 'secret_digit', 'args': {'secret': '7'}}
Tool result: The secret number is 7
=== Turn 3 ===
Thinking: Since you've already provided the secret number as 7, I can confirm that without further calculations needed.
Final Answer: 
The secret number is 7.

=== Turn 1 ===
Thinking: I need to gather some information or perform calculations that might lead me to identify the secret number. Since I don't have any initial hints or clues, I'll start by asking the secret_digit 

['\nThe secret number is 7.\n',
 '\n7\n',
 '\nThe secret number is 7.\n',
 '\nThe secret number is 7.\n',
 '\n7\n',
 '\nThe secret number is 7.\n']